In [4]:
%load_ext autoreload
%autoreload 2
%cd /home/jovyan/work

import sys
sys.path.append("/home/jovyan/work/src")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/home/jovyan/work


In [57]:
import numpy as np
import tensorflow as tf
import pickle as pkl
from sklearn.model_selection import train_test_split
from sentiment import tweet_utils
from sentiment import tweet_loader

In [15]:
unk_token = "__<UNK>__"
tweets = tweet_loader.load_all_tweets()
counter = tweet_utils.build_counter([x[0] for x in tweets])
vocab = tweet_utils.build_vocab(counter, 10, [x[0] for x in tweets], unk_token=unk_token)

__Model hyper parameters__

In [69]:
model_path = "model/sentiment.keras"
emb_dim = 128
num_epochs = 5
batch_size = 64
maxlen = 128
lstm_units = 100

In [46]:
train_tweets, test_tweets = train_test_split(tweets, test_size=0.1)
def tweets_to_X_y(vocab: dict, tweets: list, unk_token: str, maxlen: int) -> tuple:
    X = [x[0] for x in tweets]
    X = [tweet_utils.tweet_to_tensor(vocab, x[0], unk_token, maxlen) for x in tweets]
    y = [x[1] for x in tweets]
    return np.array(X).reshape(-1, maxlen), np.array(y)

In [47]:
X_train, y_train = tweets_to_X_y(vocab, train_tweets, unk_token, maxlen)
X_test, y_test = tweets_to_X_y(vocab, test_tweets, unk_token, maxlen)

In [51]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=len(vocab), output_dim=emb_dim, input_length=maxlen))
model.add(tf.keras.layers.LSTM(lstm_units))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size)
model.save(model_path)

num_correct = 0
for ind, score in enumerate(model.predict(X_test)):
    num_correct += y_test[ind] == (score >= 0.5)
print(f"Accuracy on test dataset: {num_correct / len(y_test)}")

context = {
    "model.emb_dim": 128,
    "model.num_epochs": 5,
    "model.batch_size": 64,
    "model.maxlen": 128,
    "model.lstm_units": 100,
    "model.sentiment_path": model_path,
    "data.unk": unk_token,
    "data.vocab": vocab,
    "data.counter": counter
}
pkl.dump(open("model/context.pkl"), context)

Epoch 1/5
443/443 [==============================] - 55s 123ms/step - loss: 0.3839 - accuracy: 0.8356
Epoch 2/5
443/443 [==============================] - 53s 120ms/step - loss: 0.3176 - accuracy: 0.8721
Epoch 3/5
443/443 [==============================] - 52s 118ms/step - loss: 0.2972 - accuracy: 0.8783
Epoch 4/5
443/443 [==============================] - 57s 128ms/step - loss: 0.2793 - accuracy: 0.8870
Epoch 5/5
99/99 [==============================] - 4s 31ms/step
Accuracy on test dataset: [0.87158296]


In [78]:
context = {
    "model.emb_dim": 128,
    "model.num_epochs": 5,
    "model.batch_size": 64,
    "model.maxlen": 128,
    "model.lstm_units": 100,
    "model.sentiment_path": model_path,
    "model.unk_token": unk_token,
    "data.vocab": vocab,
    "data.counter": counter
}
pkl.dump(context, open("model/context.pkl", "wb"))

# Load model

In [84]:
from sentiment import sentiment_analyzer
analyzer = sentiment_analyzer.SentimentAnalyzer()

In [87]:
analyzer.analyze("Chào bạn, mình rất thích bạn")

1/1 [==============================] - 0s 29ms/step


0.95593405